# 仅作数据分析代码记录
# 分析时间：20220429
# 记录时间：20220501
# 项目：ASV相对丰度两两组间显著性检验

### 用到的模块

In [ ]:
# 导入模块
import pandas as pd
import scipy.stats as ss

### 使用的函数：wrstest_c
### 检验的方法：秩和检验

In [ ]:
def wrstest_c(df1, columnsname):
    """
    删去把返回的df存为excel的功能

    :param df1: 处理的数据. 数据类型：dataframe
    :param columnsname: df中每一列的列名
    :return: 返回一个储存显著性检验的组名和检验p值的df

    使用这个函数，df1的提供分组信息的列的列名必须为”GroupByTimeDiet“
    需要的库：pandas scipy
    """

    dfGroup = df1.groupby('GroupByTimeDiet')  # 按'GroupByTimeDiet'分组
    KeyList = list(dfGroup.groups.keys())  # 分组组名转为列表
    i = 0
    resultWilcoxonRankSumtest = {} # 定义字典 用于储存秩和检验的结果
    while i < len(KeyList):
        # 遍历的次数 = 分组的个数
        data1 = df1[columnsname][df1.GroupByTimeDiet == KeyList[i]] # data1 = df1的"columnsname"列中KeyList[i]组的数据
        p = i + 1
        while p < len(KeyList):
            data2 = df1[columnsname][df1.GroupByTimeDiet == KeyList[p]] # data2 = df1的"columnsname"列中KeyList[p]组的数据。如果i是1，p就是2 data1和data2就是同一列里KeyList[1]组和KeyList[2]组的数据

            rWRStKey = KeyList[i] + " " + KeyList[p]  # 设置字典里每个键的名字，KeyList[i]和KeyList[p]这两个组名的组合，以空格分开
            WilcoxonRankSumtestPvalue = ss.mannwhitneyu(data1, data2)[1]  # 对data1和data2进行Wilcoxon Rank Sum test ，WilcoxonRankSumtestPvalue储存p值
            resultWilcoxonRankSumtest[rWRStKey] = WilcoxonRankSumtestPvalue  # 设置键的值为检验结果的p值
            p += 1
        i += 1
    df = pd.DataFrame([resultWilcoxonRankSumtest]) # 字典转DF
    df.index = [columnsname]  # 修改行索引为待处理的表中的列名
    return df.T

### 执行ASV相对丰度两两组间显著性检验

In [ ]:
# 读文件
df = pd.read_excel(r'D:\Work\分析项目\wxy\宏基因组与16S丰度计算\ASV丰度(加测数据)\ASV_relative_depoint.xlsx'
                    , header=0, index_col=0)
df.drop(["Taxonomy"],axis=1,inplace=True)
df1 = df.T

# 转换df1中的数据类型 ，object转float
for i in df1.columns:
    if i != "GroupByTimeDiet":
        df1[i] = df1[i].astype(float, errors = 'raise')

# 定义储存函数返回的df的列表
dflist = []

# 调用函数
for i in df1.columns:
    if i != "GroupByTimeDiet":
        dflist.append(wrstest_c(df1=df1, columnsname=i))
    else:
        print("列名为GroupByTimeDiet")

# 合并df，然后转为excel储存
dflistConcat = pd.concat(dflist, axis=1)
dflistConcat.to_excel(r'D:\Work\分析项目\wxy\宏基因组与16S丰度计算\ASV丰度(加测数据)\ASV_relative_depoint丰度变化两两显著性检验.xlsx')